My personal interpretation of the finite vs infinite risk centers around the concept of delusion. Obviously, we in "real life" know that the unconstrained profit maximizing strategy will lead to very high risk as the ecophysical theory of overfishing is well understood. The magic happens when we consider that the "risk" being talked about here isn't risk from the objective, impartial observer that we are, but from the perspective of the partial, profit-motivated agent-stakeholder system that must calculate the risk, after all. When the agent-stakeholder system places an emphasis on high omega, corresponding to high discounting of future losses (and a myopic corporate view), and a calculation of the risk that's relatively low, then that would indicate the stakeholders are highly deluded about the situation, because the actual ecophysical "risk" is really very high so they're underestimating. But when the agent-stakeholder system emphasizes a low omega (corresp to low discounting of future losses/treating the future losses almost as seriously as the present, or an ecocentric non-profit-greedy view), resulting in a calculated risk that's relatively high, that would indicate the stakeholders are "clear-eyed" and not deluded, because they perceive the unconstrained policy's results to be as seriously bad as it "actually" ecophysically is.

Let's create an automated script that would store the ratios of risk to Gt. This goes hand-in-hand with my creation of a new DecayingPreferenceModelWithLogging class that would log it.

In [2]:
import pymc as pm
import numpy as np
from sim.utils import Params
from sim.models import EulerMaruyamaDynamics, \
    RevenueModel, CostModel, RiskMitigationPolicy, \
    ProfitMaximizingPolicy, \
    LossModel, SoftmaxPreferencePrior, UniformPreferencePrior, \
    RiskModel, Model, DecayingPreferenceModel, DecayingPreferenceModelWithLogging


t_end = 100
num_points = 20
D = 100  # no Brownian motion
mc = 100
horizon = 20
P0 = 1
rho = -0.9
C0 = 1
gamma = -0.9
# maximum loss occurs when cost = 0, revenue = P0 * B_max ** (1 + rho)
B_max = 100000
l_bar = P0 * B_max ** (1 + rho)

lmbdas = np.linspace(0., 1000., 100)

dynamics = EulerMaruyamaDynamics(t_end, num_points, D, B_max)
revenue_model = RevenueModel(P0=P0, rho=rho)
cost_model = CostModel(C0=C0, gamma=gamma)
# policy = RiskMitigationPolicy(revenue_model, cost_model, lmbda=0.1)
policy = ProfitMaximizingPolicy(revenue_model, cost_model)
loss_model = LossModel()
preference_prior = SoftmaxPreferencePrior(l_bar)
# preference_prior = UniformPreferencePrior(l_bar)
risk_model = RiskModel(preference_prior)

NUM_PARAM_BATCHES = 1

for candidate_w in [0, 0.005, 0.05, 1]:

    with pm.Model() as pm_model:  # this is a pymc model and in particular the "with...as..." syntax means all assignments in this block are associated with this model's context!
        B0 = pm.Normal("B", mu=90000, sigma=2000)
        # w = pm.LogNormal('w', mu=1., sigma=0.00001)
        w = pm.Uniform('w', lower=candidate_w, upper=candidate_w)
        r = pm.Normal("r", mu=0.2, sigma=0.06)
        k = pm.Normal("k", mu=100000, sigma=10000)
        qE = pm.Uniform("qE", lower=0., upper=1.)

        samples = pm.sample_prior_predictive(samples=NUM_PARAM_BATCHES)

    p = Params(**samples.prior)

    candidate_decaying_model = DecayingPreferenceModelWithLogging(
        p,
        mc,
        dynamics,
        horizon,
        policy,
        revenue_model,
        cost_model,
        loss_model,
        risk_model,
        debug=True,
    )
    candidate_ratio = candidate_decaying_model()
    print("Candidate w =", candidate_w, "has ratio of Rt_sim to minimum Gt:", candidate_ratio)

Sampling: [B, k, qE, r, w]


Sampling: [B, k, qE, r, w]


sample_mean [0.] - entropy [-8.21400435] = RISK: [8.2140045]
sample_mean [0.] - entropy [-7.71389175] = RISK: [7.713892]
sample_mean [0.] - entropy [-7.3705435] = RISK: [7.3705435]
sample_mean [0.] - entropy [-6.97561808] = RISK: [6.975618]
sample_mean [0.] - entropy [-6.60594598] = RISK: [6.605946]
sample_mean [0.] - entropy [-6.40959222] = RISK: [6.409592]
sample_mean [0.] - entropy [-6.09879995] = RISK: [6.0988]
sample_mean [0.] - entropy [-5.93186122] = RISK: [5.9318614]
sample_mean [0.] - entropy [-5.6375446] = RISK: [5.6375446]
sample_mean [0.] - entropy [-5.36889456] = RISK: [5.3688946]
sample_mean [0.] - entropy [-5.25458525] = RISK: [5.2545853]
sample_mean [0.] - entropy [-5.02058157] = RISK: [5.0205817]
sample_mean [0.] - entropy [-4.79237284] = RISK: [4.7923727]
sample_mean [0.] - entropy [-4.54702465] = RISK: [4.5470247]
sample_mean [0.] - entropy [-4.40395168] = RISK: [4.4039516]
sample_mean [0.] - entropy [-4.21610569] = RISK: [4.2161055]
sample_mean [0.] - entropy [-4.00

Sampling: [B, k, qE, r, w]
Sampling: [B, k, qE, r, w]


sample_mean [0.] - entropy [-4.66940211] = RISK: [4.669402]

end plan

Rt_sim: [122.89682] and min_Gt: [4.669402]
Candidate w = 0.005 has ratio of Rt_sim to minimum Gt: 26.319605
sample_mean [0.] - entropy [-8.3109227] = RISK: [8.310923]
sample_mean [0.] - entropy [-7.93635522] = RISK: [7.936355]
sample_mean [0.] - entropy [-7.70384341] = RISK: [7.7038436]
sample_mean [0.] - entropy [-7.39541545] = RISK: [7.3954153]
sample_mean [0.] - entropy [-7.23749544] = RISK: [7.2374954]
sample_mean [0.] - entropy [-6.99962102] = RISK: [6.999621]
sample_mean [0.] - entropy [-6.88187652] = RISK: [6.8818765]
sample_mean [0.] - entropy [-6.80352002] = RISK: [6.80352]
sample_mean [0.] - entropy [-6.61943859] = RISK: [6.6194386]
sample_mean [0.] - entropy [-6.42589008] = RISK: [6.42589]
sample_mean [0.] - entropy [-6.40076906] = RISK: [6.400769]
sample_mean [0.] - entropy [-6.33752343] = RISK: [6.3375235]
sample_mean [0.] - entropy [-6.18292371] = RISK: [6.182924]
sample_mean [0.] - entropy [-6.1466936